### Custom MBA courses RAG project
---

In [1]:
# Go one level up in the directories hierarchy to access src directory and codes
import sys
import os
# Add project root to Python path
project_root = os.path.abspath("..")  # go one level up from notebooks/
sys.path.append(project_root)

In [2]:
# Setup necessary models for routing, chatting and embedding
from core.config.llm_setup import LLMsetups

router_llm = LLMsetups.ROUTER_LLM
chat_llm = LLMsetups.CHAT_LLM
embed_model = LLMsetups.EMBED_MODEL

c:\Users\Sanzhar\workspace\github.com\kaydarovv\ai_agent_learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Test the RagSystem class defined

In [3]:
# Let's test our RAG module defined in class to see how well it is refactored
from core.src.rag_workflow import RagChatWorkflow

import nest_asyncio
nest_asyncio.apply()

rag_chat = RagChatWorkflow()

2025-12-17 09:34:30,563 - WARNING - Ignoring wrong pointing object 9 0 (offset 0)
2025-12-17 09:34:30,564 - WARNING - Ignoring wrong pointing object 20 0 (offset 0)
2025-12-17 09:34:30,564 - WARNING - Ignoring wrong pointing object 22 0 (offset 0)
2025-12-17 09:34:30,565 - WARNING - Ignoring wrong pointing object 30 0 (offset 0)
2025-12-17 09:34:30,565 - WARNING - Ignoring wrong pointing object 36 0 (offset 0)
2025-12-17 09:34:30,566 - WARNING - Ignoring wrong pointing object 49 0 (offset 0)
2025-12-17 09:34:30,567 - WARNING - Ignoring wrong pointing object 55 0 (offset 0)
2025-12-17 09:34:30,567 - WARNING - Ignoring wrong pointing object 82 0 (offset 0)
2025-12-17 09:34:30,568 - WARNING - Ignoring wrong pointing object 84 0 (offset 0)
2025-12-17 09:34:30,569 - WARNING - Ignoring wrong pointing object 95 0 (offset 0)
2025-12-17 09:34:30,569 - WARNING - Ignoring wrong pointing object 112 0 (offset 0)
2025-12-17 09:34:30,570 - WARNING - Ignoring wrong pointing object 115 0 (offset 0)
202

In [4]:
# Let's test the query and memory of the RAG system
user_name = "SomeNewUSer"
user_id = "test_user_id0"
user_query = "What are the differences between Financial and Managerial accounting?"

rag_response = await rag_chat.run(
    user_query = user_query,
    user_name = user_name, 
    user_id = user_id
)
print(rag_response)

2025-12-17 09:36:59,774 - INFO - AFC is enabled with max remote calls: 10.
2025-12-17 09:37:01,112 - INFO - Selecting retriever 0: This choice focuses on financial accounting, describing its goal, components, and target audience (users rather than prepares). While it doesn't directly contrast with managerial accounting, it provides a detailed description of one side of the comparison..
2025-12-17 09:37:01,141 - INFO - AFC is enabled with max remote calls: 10.
2025-12-17 09:37:01 | INFO     | core.src.chat_engine_registry:get_or_create_chat_engine:23 - Creating a new chat engine for user test_user_id0
2025-12-17 09:37:01,647 - INFO - AFC is enabled with max remote calls: 10.


SomeNewUSer, the provided context outlines the following distinctions between financial and managerial accounting:

**Financial Accounting:**
*   **Rules:** Prepared under external rules such as IFRS and US-GAAP.
*   **Auditing:** Subject to audits by firms like KPMG, EY, Deloitte, and PwC.
*   **Users:** Primarily serves external stakeholders including shareholders, creditors, tax authorities, labor unions, and employees.
*   **Units of Measure:** Uses monetary units (e.g., Tenge, USD).
*   **Focus:** Considered "external" accounting.
*   **Reports:** Produces financial statements (e.g., Balance sheet, 10K-Form).

**Managerial Accounting:**
*   **Rules:** Prepared under internal guidelines.
*   **Auditing:** Not audited.
*   **Users:** Primarily serves internal management, such as c-suite executives (CEO, CFO).
*   **Units of Measure:** Utilizes both monetary and non-monetary units (e.g., hours per product).
*   **Focus:** Considered "internal" accounting.
*   **Reports:** Includes bu

In [5]:
# Let's test the new user
user_name2 = "SomeNewUSer"
user_id2 = "test_user_id2"
user_query2 = "Do you think that people would have money now or later?"

rag_response2 = await rag_chat.run(
    user_query = user_query2,
    user_name = user_name2, 
    user_id = user_id2
)
print(rag_response2)

2025-12-17 09:37:13,101 - INFO - AFC is enabled with max remote calls: 10.
2025-12-17 09:37:14,092 - INFO - Selecting retriever 1: This choice explicitly states '1 Time is Money (Impatience Principle) - All else equal, individuals prefer money now to later.' which directly addresses the question of whether people would have money now or later..
2025-12-17 09:37:14,119 - INFO - AFC is enabled with max remote calls: 10.
2025-12-17 09:37:14 | INFO     | core.src.chat_engine_registry:get_or_create_chat_engine:23 - Creating a new chat engine for user test_user_id2
2025-12-17 09:37:14,530 - INFO - AFC is enabled with max remote calls: 10.


SomeNewUSer, the provided context does not contain information to determine whether people would have money now or later. It only introduces the concept of the time value of money and poses questions about the future value of money.


In [8]:
rag_chat.chat_engine_registry.chat_engines_cached["test_user_id0"]._memory.get()

[ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="\n            You are a helpful academic assistant specialized in MBA courses for Master of Engineering Management (MEM) students.\n\n            Address the user by their name: (Note: user's name will be provided to you alongside each user question).\n\n            Your task is to answer the user's question using ONLY the information provided in the context. \n            (Note: The specific context will be provided to you alongside each user question).\n\n            Rules:\n            - Answer the question clearly, accurately, and concisely.\n            - Use ONLY the provided context to construct your answer.\n            - Do NOT use outside knowledge, assumptions, or general world knowledge.\n            - Do NOT invent facts or fill in missing information.\n            - If the context does not contain enough information to answer the question, explicitly state that t